In [1]:
!mkdir -p /scratch/sagarsj42/torch-cache
!mkdir -p /scratch/sagarsj42/transformers
import os
os.chdir('/scratch/sagarsj42')
os.environ['TORCH_HOME'] = '/scratch/sagarsj42/torch-cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/sagarsj42/transformers'

In [103]:
from functools import partial

import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

from transformers import XLMRobertaTokenizer, XLMRobertaModel

In [3]:
!scp sagarsj42@ada:/share1/sagarsj42/semeval-2022-task-8-train-df.csv .
!scp sagarsj42@ada:/share1/sagarsj42/semeval-2022-task-8-eval-df.csv .

semeval-2022-task-8-train-df.csv              100%   27MB  27.3MB/s   00:00    
semeval-2022-task-8-eval-df.csv               100%   27MB  27.4MB/s   00:01    


In [37]:
train_df = pd.read_csv('semeval8-train-sss.csv', index_col=0)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4467 entries, 3913 to 735
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   pair_id             4467 non-null   object 
 1   score               4467 non-null   float64
 2   id_1                4467 non-null   int64  
 3   title_1             4427 non-null   object 
 4   text_1              4436 non-null   object 
 5   keywords_1          4467 non-null   object 
 6   meta_keywords_1     4467 non-null   object 
 7   tags_1              4467 non-null   object 
 8   summary_1           11 non-null     object 
 9   meta_description_1  3720 non-null   object 
 10  meta_lang_1         3869 non-null   object 
 11  id_2                4467 non-null   int64  
 12  title_2             4433 non-null   object 
 13  text_2              4426 non-null   object 
 14  keywords_2          4467 non-null   object 
 15  meta_keywords_2     4467 non-null   object 
 16  tags

In [38]:
train_df.head()

,pair_id,score,id_1,title_1,text_1,keywords_1,meta_keywords_1,tags_1,summary_1,meta_description_1,meta_lang_1,id_2,title_2,text_2,keywords_2,meta_keywords_2,tags_2,summary_2,meta_description_2,meta_lang_2
3913,1593242729_1506953769,4.0,1593242729,Brände: Stall mit 1000 Schweinen brennt ab,Schöppingen. Ein Schweinestall mit rund 1000 T...,[],"['Brände', 'Tiere', 'NRW', 'Nordrhein-Westfale...",[],NaN,Ein Schweinestall mit rund 1000 Tieren ist am ...,NaN,1506953769,Neue Flüchtlingsunterkunft des Landes in Dorsten,In die neue Landesunterkunft sollen vor allem ...,[],[''],[],NaN,NaN,de
2962,1537050788_1536175708,2.0,1537050788,NaN,"Przykro nam, ale z adresu IP z którego się łąc...",[],[''],[],NaN,NaN,pl,1536175708,NaN,"Przykro nam, ale z adresu IP z którego się łąc...",[],[''],[],NaN,NaN,pl
2605,1544221535_1544175685,1.0,1544221535,Diese Raupe isst Plastik und pinkelt Alkohol,Die Raupen der Grossen Wachsmotte verbringen i...,[],"['20minuten.ch', 'www.20minuten.ch', '20 Minut...",[],NaN,Die Raupe der Grossen Wachsmotte ist in der La...,de,1544175685,Winziger plastikessender Wurm unter wissenscha...,"Die Fähigkeit der Würmer, sich von Polyethylen...",[],"['Winziger Wachswurm', 'Raupe', 'besondere Bak...","['Bakterien', 'Plastik', 'fressen', 'Würmer', ...",NaN,<p>Forscher von der Brandon University in Kana...,de
3075,1592201688_1617897198,3.0,1592201688,Caddeyi saksafon sesiyle şenlendirdi,Caddeyi saksafon sesiyle şenlendirdi\n\nKırkla...,[],"['İstanbul Caddesi', 'Kırklareli', 'Lüleburgaz...",[],NaN,Caddeyi saksafon sesiyle şenlendirdi Kırklarel...,tr,1617897198,Kısıtlama sonrası korkutan rehavet görüntüsü,Kısıtlama sonrası korkutan rehavet görüntüsü K...,[],"['İstanbul Caddesi', 'Kırklareli', 'Lüleburgaz...",[],NaN,Kısıtlama sonrası korkutan rehavet görüntüsü -...,tr
501,1484037661_1485647004,1.0,1484037661,Giuliani Says He’s Prepared to ‘Do Demonstrati...,Rudy Giuliani is prepared to do more than just...,[],"['Donald J. Trump', 'Ukraine', 'Rudy Giuliani'...",[],NaN,He’s also prepared to “try the case” and “give...,en,1485647004,VERY interesting comment from Giuliani (raw)…,What the hell just happened? pic.twitter.com/Q...,[],[''],[],NaN,America's Front Page For Political News,en


In [39]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
encoder = XLMRobertaModel.from_pretrained('xlm-roberta-base')
encoder

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


XLMRobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): 

In [40]:
sample = train_df.iloc[0]
sample.keys()

Index(['pair_id', 'score', 'id_1', 'title_1', 'text_1', 'keywords_1',
       'meta_keywords_1', 'tags_1', 'summary_1', 'meta_description_1',
       'meta_lang_1', 'id_2', 'title_2', 'text_2', 'keywords_2',
       'meta_keywords_2', 'tags_2', 'summary_2', 'meta_description_2',
       'meta_lang_2'],
      dtype='object')

In [41]:
sample_dict = train_df.iloc[0][['pair_id', 'text_1', 'text_2', 'score']].to_dict()
sample_dict['text_1'] = sample_dict['text_1'].lower().strip()
sample_dict['text_2'] = sample_dict['text_2'].lower().strip()
sample_dict['score'] = round(sample_dict['score']) - 1
sample_dict

{'pair_id': '1593242729_1506953769',
 'text_1': 'schöppingen. ein schweinestall mit rund 1000 tieren ist am mittwochmorgen in schöppingen im münsterland abgebrannt.\n\netwa 900 schweine verendeten in den flammen, 100 befanden sich am nachmittag noch mit schweren verbrennungen in dem einsturzgefährdeten gebäude, wie ein sprecher des kreises borken sagte. die überlebenden tiere müssten vermutlich durch das kreisveterinäramt getötet werden, da sie aufgrund der verbrennungen und rauchvergiftungen „keine hohen überlebenschancen haben“, erklärte ein polizeisprecher.\n\nein 47 jahre alter landwirt hatte nach polizeiangaben noch versucht, die flammen zu löschen. er habe durch die starke rauchentwicklung selbst eine rauchvergiftung erlitten und sei ins krankenhaus gekommen. die polizei vermutet nach ersten ermittlungen einen technischen defekt als brandursache. der schaden beläuft sich demnach auf etwa 900 000 euro. angrenzende gebäude wurden nicht beschädigt.',
 'text_2': 'in die neue landesun

In [42]:
text = sample['text_1']
text_tok = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
text_tok['input_ids'].shape, text_tok['attention_mask'].shape

(torch.Size([1, 222]), torch.Size([1, 222]))

In [43]:
text_enc = encoder(**text_tok)
text_enc.last_hidden_state.shape, text_enc.pooler_output.shape

(torch.Size([1, 222, 768]), torch.Size([1, 768]))

In [100]:
class SimilarityClassifier(nn.Module):
    def __init__(self, encoder, embed_size=768, hidden_size=512, n_classes=4):
        super(SimilarityClassifier, self).__init__()

        self.encoder = encoder
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.n_classes = n_classes
        
        self.linear1 = nn.Linear(self.embed_size, self.hidden_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        self.linear2 = nn.Linear(self.hidden_size, self.n_classes)
        self.activation2 = nn.Softmax(dim=-1)

    def common_compute(self, x):
        x = self.encoder(**x).pooler_output
        x = self.linear1(x)
        x = self.activation1(x)

        return x
    
    def forward(self, x1, x2):
        x1 = self.common_compute(x1)
        x2 = self.common_compute(x2)
        x = torch.abs(x1 - x2)
        x = self.linear2(x)
        x = self.activation2(x)

        return x

In [101]:
model = SimilarityClassifier(encoder, embed_size=768, hidden_size=512, n_classes=4)
model

SimilarityClassifier(
  (encoder): XLMRobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [90]:
class MultilingualNewsSimDataset(Dataset):
    def __init__(self, df):
        super(MultilingualNewsSimDataset, self).__init__()
        self.df = df

    def __getitem__(self, idx):
        return self.df.iloc[idx][['pair_id', 'text_1', 'text_2', 'score']].to_dict()
        
    def __len__(self):
        return self.df.shape[0]

In [91]:
train_dataset = MultilingualNewsSimDataset(train_df)
len(train_dataset), train_dataset[33]

(4467,
 {'pair_id': '1558576281_1558746898',
  'text_1': "İşsizlik maaşı şartları nelerdir? Soruları işsizlik maaşı alacak olan işçi vatandaşlar tarafından sorgulanıyor. İş yerinden ayrılmış, sözleşmesi feshedilmiş çalışanlar İŞKUR üzerinden başvuru formu doldurarak işsizlik maaşını almaya hak kazanıyorlar. Peki, işsizlik maaşı başvuru nasıl yapılır?\n\nİŞSİZLİK MAAŞI ŞARTLARI NELERDİR?\n\nİşsizlik maaşı alabilmek için, son 3 yılda en az 600 gün prim ödenmesi ve son 120 gün hizmet akdiyle çalışılması gerekiyor. İŞKUR'a işsizlik başvurusunun, iş akdinin feshinden itibaren 30 gün içinde yapılması gerekiyor.\n\n\n\nİŞSİZLİK BAŞVURU VE MAAŞI SORGULAMA\n\nİşsizlik maaşı ile ilgili sorgulamalarınız için E Devlet Kapısı'nı kullanabilirsiniz.\n\n(Başvuru için: https://www.turkiye.gov.tr/issizlik-odenegi-basvurusu)\n\n(İşsizlik ödeneği sorgulamak için: https://www.turkiye.gov.tr/issizlik-odenegi-odemesi)\n\nNE KADAR SÜRE İŞSİZLİK MAAŞI ALINABİLİR?\n\nİŞKUR'un resmi internet sitesinde yaptığı bi

In [92]:
dev_df = pd.read_csv('semeval8-dev.csv')
dev_dataset = MultilingualNewsSimDataset(dev_df)
len(dev_dataset), dev_dataset[21]

(497,
 {'pair_id': '1484038753_1484148859',
  'text_1': 'California is poised to lose a congressional seat for the first time in its history as a state, based on U.S. Census Bureau population estimates released Monday that showed the nation’s growth continued to slow in 2019.\n\nSome 27 states and the District of Columbia lost residents through net domestic migration between 2018 and 2019, the new census data show.\n\nAbout 203,000 people left California in that period, a result of the state’s shifting migration patterns and economic strains that are making it harder to afford living here. New York, Illinois, New Jersey, Massachusetts and Louisiana also saw large losses to other states.\n\nCalifornia’s potential loss in reapportionment, which will be determined by next year’s census count, would drop the state’s number of seats in the U.S. House of Representatives from 53 to 52, said William Frey, a senior fellow at the Brookings Institution.\n\nAdvertisement\n\n“It’s got a lot to do w

In [93]:
def collate_fn(batch, tokenizer, num_classes=4, type='train'):
    texts_1, texts_2, scores = list(), list(), list()
    for sample in batch:
        text1 = str(sample['text_1']).lower().strip()
        text2 = str(sample['text_2']).lower().strip()

        score_indx = round(sample['score']) - 1
        score = torch.zeros((1, num_classes))
        score[0, score_indx] = 1
        
        texts_1.append(text1)
        texts_2.append(text2)
        scores.append(score)

    texts_1 = tokenizer(texts_1, truncation=True, padding=True, return_tensors='pt')
    texts_2 = tokenizer(texts_2, truncation=True, padding=True, return_tensors='pt')
    scores = torch.cat(scores, dim=0)

    return (texts_1, texts_2) if type == 'test' else (texts_1, texts_2, scores)

In [94]:
collate_partial = partial(collate_fn, tokenizer=tokenizer, num_classes=4, type='train')
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_partial)

batch = next(iter(train_dataloader))
batch

({'input_ids': tensor([[     0,  16188,  80059,  ...,  80939,      5,      2],
         [     0,    359,  27661,  ...,      1,      1,      1],
         [     0,     95,   5800,  ...,      1,      1,      1],
         [     0, 121477,   1681,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]])},
 {'input_ids': tensor([[     0,    388,  61909,  ..., 149976,     23,      2],
         [     0,  27498,  38126,  ...,      1,      1,      1],
         [     0,   1642,  15234,  ...,      1,      1,      1],
         [     0, 121477,   1681,  ...,  14810, 119477,      2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]])},
 tensor([[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [1., 0., 0., 0.]]))

In [102]:
x1, x2, scores = batch
op = model(x1, x2)
op

tensor([[0.2470, 0.2540, 0.2488, 0.2502],
        [0.2465, 0.2543, 0.2493, 0.2499],
        [0.2470, 0.2541, 0.2486, 0.2503],
        [0.2467, 0.2541, 0.2488, 0.2504]], grad_fn=<SoftmaxBackward0>)